In [1]:
!pip install tensorflow-model-optimization
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 994.7 kB/s eta 0:00:00


In [2]:
# Load the Oxford Flowers 102 dataset
dataset, info = tfds.load("oxford_flowers102", as_supervised=True, with_info=True)
num_classes = info.features['label'].num_classes

# Preprocess function
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalize
    return image, label

# Prepare train and validation datasets
train_data = dataset['train'].map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_data = dataset['validation'].map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.CM0ZSI_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.CM0ZSI_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [3]:
# Define a MobileNetV2-based CNN model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the baseline model
model.fit(train_data, validation_data=val_data, epochs=5)


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/5
32/32 [==============================] - 22s 154ms/step - loss: 4.2894 - accuracy: 0.1078 - val_loss: 3.0834 - val_accuracy: 0.3569
Epoch 2/5
32/32 [==============================] - 6s 125ms/step - loss: 2.1378 - accuracy: 0.6520 - val_loss: 2.0491 - val_accuracy: 0.6137
Epoch 3/5
32/32 [==============================] - 6s 106ms/step - loss: 1.1513 - accuracy: 0.9000 - val_loss: 1.5747 - val_accuracy: 0.7029
Epoch 4/5
32/32 [==============================] - 6s 123ms/step - loss: 0.7077 - accuracy: 0.9480 - val_loss: 1.3373 - val_accuracy: 0.7431
Epoch 5/5
32/32 [==============================] - 6s 137ms/step - loss: 0.4777 - accuracy: 0.9784 - val_loss: 1.2046 - val_accuracy: 0.7569


In [4]:
# Apply random pruning
def apply_random_pruning(model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.5,
            begin_step=0,
            end_step=1000
        )
    }

    pruned_model = prune_low_magnitude(model, **pruning_params)
    return pruned_model

# Apply pruning and recompile
pruned_model = apply_random_pruning(model)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Add the UpdatePruningStep callback
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

# Fine-tune pruned model
pruned_model.fit(train_data, validation_data=val_data, epochs=5, callbacks=callbacks)

Epoch 1/5
32/32 [==============================] - 43s 223ms/step - loss: 0.4086 - accuracy: 0.9716 - val_loss: 1.0704 - val_accuracy: 0.7578
Epoch 2/5
32/32 [==============================] - 6s 127ms/step - loss: 0.2019 - accuracy: 0.9922 - val_loss: 0.9388 - val_accuracy: 0.7814
Epoch 3/5
32/32 [==============================] - 6s 132ms/step - loss: 0.1246 - accuracy: 0.9990 - val_loss: 0.8879 - val_accuracy: 0.7843
Epoch 4/5
32/32 [==============================] - 10s 226ms/step - loss: 1.1379 - accuracy: 0.7696 - val_loss: 1.5630 - val_accuracy: 0.6373
Epoch 5/5
32/32 [==============================] - 8s 169ms/step - loss: 0.7132 - accuracy: 0.8931 - val_loss: 1.3851 - val_accuracy: 0.6775


In [5]:
# Evaluate pruned model
_, pruned_acc = pruned_model.evaluate(val_data)
print(f"Pruned Model Accuracy: {pruned_acc:.4f}")

32/32 [==============================] - 2s 55ms/step - loss: 1.3851 - accuracy: 0.6775
Pruned Model Accuracy: 0.6775
